# Use Lax-Friedrichs, Lax-Wedroff, and MacCormack

In [88]:
import numpy
from matplotlib import pyplot
import ipywidgets

%matplotlib inline

In [170]:
def rho_red_light(x, rho_max):
    """Compute red light initial conditions with the shock
    
    x : location on the road
    rho_max : maximum traffic density
    
    return rho : initial car density on the road"""
    
    rho = rho_max * numpy.ones_like(x)
    # or rho = numpy.full_like(x, rho_max)
    mask = numpy.where(x < 3.0)
    rho[mask] = 0.45 * rho_max
    # or rho[mask] /= 2
    
    return rho

In [171]:
#Set parameters bruh
nx = 81 #number of locations on the road
L = 4.0 #length of the road
dx = L / (nx -1) #distance between locations
nt = 40 #number of time steps
sigma = 1.0
dt = sigma * dx / u_max #time step size
rho_max = 10.0 #maximum allowable traffic density
u_max = 1.0 #maximum speed of traffic

#Get the road locations
x = numpy.linspace(0.0, L, num=nx)

#Computer initial traffic density
rho0 = rho_red_light(x, rho_max)

In [172]:
def plot(n, x, rho_hist):
    """Plot solution at a given time and compare to initial condtion"""
    
    pyplot.figure()
    pyplot.plot(x, rho_hist[0], label = 'Initial')
    pyplot.plot(x, rho_hist[n], label = 'Current')
    pyplot.grid()
    pyplot.legend()
    pyplot.xlim(0.0, 4.0)
    pyplot.ylim(4.0 , 11.0)
    pyplot.xlabel('Length on road')
    pyplot.ylabel('Density of cars')
    pyplot.show()

In [173]:
rho_hist = [rho0.copy()]

In [174]:
def traffic_flux(rho, u_max, rho_max):
    """Compute the traffic flux F = V * rho
    
    rho : traffic density
    u_max : max speed
    rho_max : max density
    
    return F"""
    
    F = rho * u_max * (1.0 - rho / rho_max)
    
    return F

In [175]:
def traffic_speed(rho, u_max, rho_max):
    """Compute the traffic speed 
    
    rho : traffic density
    u_max : max speed
    rho_max : max density
    
    return F"""
    
    F = u_max * (1.0 - rho / rho_max)
    
    return F

In [176]:
def interactive_plot(x, rho_hist):
    nt = len(rho_hist) - 1
    nt_slider = ipywidgets.IntSlider(value=0, min=0, max=nt, step=1,
                                    description='Time Step')
    w = ipywidgets.interactive(plot, n=nt_slider,
                              x=ipywidgets.fixed(x),
                              rho_hist=ipywidgets.fixed(rho_hist))
    
    return w

In [178]:
interactive_plot(x, rho_hist)

interactive(children=(IntSlider(value=0, description='Time Step', max=0), Output()), _dom_classes=('widget-int…

In [179]:
def lax_friedrichs(rho0, nt, dt, dx, rho_max, u_max):
    """Compute traffic density using the lax_friedrichs method"""
    rho_hist = [rho0.copy()]
    rho = rho0.copy()
    for n in range(nt):
        F = traffic_flux(rho, u_max, rho_max) #compute the traffic flux
        rho[1:-1] = (0.5 * (rho[2:] + rho[:-2]) - 
        0.5 * dt / dx * (F[2:] - F[:-2]))
        rho_hist.append(rho.copy())
    
    return rho_hist

In [180]:
rho_hist = lax_friedrichs(rho0, nt, dt, dx, rho_max, u_max)

In [181]:
interactive_plot(x, rho_hist)

interactive(children=(IntSlider(value=0, description='Time Step', max=40), Output()), _dom_classes=('widget-in…

In [182]:
sigma = 1.0
dt = sigma * dx / u_max
nt = 40
rho_hist = lax_friedrichs(rho0, nt, dt, dx, rho_max, u_max)
interactive_plot(x, rho_hist)

interactive(children=(IntSlider(value=0, description='Time Step', max=40), Output()), _dom_classes=('widget-in…

In [183]:
def traffic_jacobian(rho, u_max, rho_max):
    """Computes the traffic jacobian"""
    
    J = u_max * (1.0 - 2.0 * rho / rho_max)
    
    return J

In [184]:
def lax_wendroff(rho0, nt, dt, dx, rho_max, u_max):
    """Computes traffic density using Lax_Wendroff Technique"""
    rho_hist = [rho0.copy()]
    rho = rho0.copy()
    for n in range(nt):
        F = traffic_flux(rho, u_max, rho_max)
        J = traffic_jacobian(rho, u_max, rho_max)
        rho[1:-1] = (rho[1:-1] - 0.5 * dt / dx * (F[2:] - F[:-2])
                     + 0.25 * dt**2 / dx**2 * ((J[1:-1] + J[2:]) *
                                                   (F[2:] - F[1:-1]) -
                                                   (J[:-2] + J[1:-1]) *
                                                   (F[1:-1] - F[:-2])))
        rho_hist.append(rho.copy())
    
    return rho_hist
                                                   

In [185]:
rho_hist = lax_wendroff(rho0, nt, dt, dx, rho_max, u_max)

In [186]:
interactive_plot(x, rho_hist)

interactive(children=(IntSlider(value=0, description='Time Step', max=40), Output()), _dom_classes=('widget-in…

In [187]:
def maccormack(rho0, nt, dt, dx, rho_max, u_max):
    """Computes traffic density utilizing the MacCormack method"""
    
    rho_hist = [rho0.copy()]
    rho = rho0.copy()
    rho_temp = rho.copy()
    for n in range(nt):
        F = traffic_flux(rho, u_max, rho_max)
        rho_temp[1:-1] = rho[1:-1] - dt / dx * (F[2:] - F[1:-1])
        F = traffic_flux(rho_temp, u_max, rho_max)
        rho[1:-1] = 0.5 * ((rho[1:-1] + rho_temp[1:-1]) - 
                           dt / dx * (F[1:-1] - F[:-2]))
                     
        rho_hist.append(rho.copy())
        
    return rho_hist
    

In [188]:
rho_hist = maccormack(rho0, nt, dt, dx, rho_max, u_max)

In [189]:
interactive_plot(x, rho_hist)

interactive(children=(IntSlider(value=0, description='Time Step', max=40), Output()), _dom_classes=('widget-in…

# Even with added "instability" via p_init < p_max/2, all three methods work very well.